In [1]:
import pandas as pd
import numpy as np
import json
import ast
from tqdm import tqdm

In [3]:
# Asegúrate de ajustar la ruta al archivo según sea necesario
df_clientes = pd.read_csv('clientes_5_o_mas_pedidos.csv')
print(f"Datos de clientes cargados: {df_clientes.shape[0]} registros")
print(df_clientes.head(2))

# Cargar el JSON de items y sus vectores
with open('items_procesados_utf8.json', 'r') as f:
    items_data = json.load(f)
print(f"Datos de items cargados: {len(items_data)} items")
print(items_data[0].keys())

Datos de clientes cargados: 129007 registros
                         FF_CUSTOMER_ID  order_count  \
0  000097e7-b84f-4517-a8a7-59faa7d4c7ef            5   
1  0001595a-e2f7-4578-8dc9-c0884a667e38           30   

                                              orders  
0  [{'order_id': 'csbkab', 'cart': "{'ITEM_B1': 1...  
1  [{'order_id': 'efrr2x', 'cart': "{'ITEM_S1': 1...  
Datos de items cargados: 109 items
dict_keys(['Item', 'itemName', 'ingredientes', 'ingredientes_vector', 'itemName_vector'])


In [4]:
# Crear un diccionario para mapear cada item a su vector
item_to_vector = {}
vector_length = len(items_data[0]["ingredientes_vector"])
print(f"Longitud de los vectores de ingredientes: {vector_length}")

for item in items_data:
    item_id = item["Item"]
    vector = item["ingredientes_vector"]
    item_to_vector[item_id] = vector

print(f"Mapeo de items a vectores creado para {len(item_to_vector)} items")

Longitud de los vectores de ingredientes: 342
Mapeo de items a vectores creado para 109 items


In [5]:
def parse_cart(cart_str):
    """
    Convierte el string del carrito en un diccionario
    """
    try:
        return ast.literal_eval(cart_str)
    except (SyntaxError, ValueError):
        print(f"Error al parsear: {cart_str}")
        return {}

In [6]:
def get_item_vectors(cart, item_to_vector):
    """
    Obtiene los vectores para cada item en el carrito
    """
    items_vectors = []
    for item_id, quantity in cart.items():
        base_item = item_id.split('_')[0] + '_' + item_id.split('_')[1]
        if base_item in item_to_vector:
            items_vectors.append(item_to_vector[base_item])
        else:
            print(f"Item no encontrado: {base_item}")
    return items_vectors

In [7]:
def combine_vectors(vectors):
    """
    Combina múltiples vectores en uno solo, 
    donde cualquier valor positivo se convierte en 1
    """
    if not vectors:
        return [0] * vector_length
    
    combined = np.sum(vectors, axis=0)
    return [1 if x > 0 else 0 for x in combined]

In [8]:
result_df = pd.DataFrame(columns=['FF_CUSTOMER_ID', 'orders'])

In [9]:
# Procesar cada cliente
for _, row in tqdm(df_clientes.iterrows(), total=df_clientes.shape[0]):
    customer_id = row['FF_CUSTOMER_ID']
    orders_list = ast.literal_eval(row['orders']) if isinstance(row['orders'], str) else row['orders']
    
    # Lista para almacenar los vectores binarios de cada orden
    order_vectors = []
    
    for order in orders_list:
        cart_str = order['cart']
        cart_dict = parse_cart(cart_str)
        
        # Obtener vectores para los items en el carrito
        item_vectors = get_item_vectors(cart_dict, item_to_vector)
        
        # Combinar los vectores en uno solo
        order_vector = combine_vectors(item_vectors)
        order_vectors.append(order_vector)
    
    # Añadir el resultado al DataFrame
    result_df = result_df._append({
        'FF_CUSTOMER_ID': customer_id,
        'orders': order_vectors
    }, ignore_index=True)

print(f"Datos transformados: {result_df.shape[0]} clientes procesados")
print(result_df.head(2))

100%|██████████████████████████████████| 129007/129007 [09:32<00:00, 225.46it/s]

Datos transformados: 129007 clientes procesados
                         FF_CUSTOMER_ID  \
0  000097e7-b84f-4517-a8a7-59faa7d4c7ef   
1  0001595a-e2f7-4578-8dc9-c0884a667e38   

                                              orders  
0  [[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...  
1  [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...  


In [10]:
# Guardar el DataFrame resultante
result_df.to_csv('clientes_vectores_binarios.csv', index=False)
print("Datos guardados exitosamente en 'clientes_vectores_binarios.csv'")

Datos guardados exitosamente en 'clientes_vectores_binarios.csv'


In [11]:
# Mostrar un ejemplo de los datos de un cliente
if result_df.shape[0] > 0:
    sample_customer = result_df.iloc[0]
    print("\nEjemplo de cliente:")
    print(f"ID: {sample_customer['FF_CUSTOMER_ID']}")
    print(f"Número de órdenes: {len(sample_customer['orders'])}")
    
    if len(sample_customer['orders']) > 0:
        print(f"Ejemplo de vector de la primera orden (primeros 10 elementos): {sample_customer['orders'][0][:10]}")
        print(f"Suma de elementos en el vector: {sum(sample_customer['orders'][0])}")
        print(f"Número de ingredientes representados: {sum(sample_customer['orders'][0])}")


Ejemplo de cliente:
ID: 000097e7-b84f-4517-a8a7-59faa7d4c7ef
Número de órdenes: 5
Ejemplo de vector de la primera orden (primeros 10 elementos): [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
Suma de elementos en el vector: 9
Número de ingredientes representados: 9


In [13]:
result_df.iloc[0]

FF_CUSTOMER_ID                 000097e7-b84f-4517-a8a7-59faa7d4c7ef
orders            [[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...
Name: 0, dtype: object